# Imports

In [44]:
import os
import sys
import inspect
import glob

import torch
from PIL import Image
import torchvision.transforms as transforms

sys.path.insert(0,'..')
import process_section
import clipscore

from torchmetrics.multimodal import CLIPScore

# Existing clipscore implementation

In [39]:
# Change num_workers to 0 to account for local issue

def process_images(image_dir, model, device):
    image_paths = glob.glob(f'{image_dir}/*/*.jpg')

    # a dictionary
    image_feats_lookup = clipscore.extract_all_images(
        image_paths, model, device, batch_size=64, num_workers=0)

    return image_feats_lookup

In [64]:
model, device = clipscore.get_clip_mdl()
imgdir = 'imgdir'
image_set = glob.glob(f'{imgdir}/*/*.jpg')
image_feats_lookup = process_images(imgdir, model, device)
segments = [('s1', 'This is a meeting'), ('s2', 'This is a python'), ('s1', 'This is Brazil'), ('s2', 'This is the Earth'), ('s1', 'This is skiing'), ('s2', 'This is a cat')]
result = process_section.calc_clip(segments, image_set, image_feats_lookup, model, device)

/Users/sterdam/cterdam/11711/photobook-full-listener/preprocess/verify_clipscore/../clipscore.py:136: UserWarning: CLIP runs in full float32 on CPU. Results in paper were computed on GPU, which uses float16. If you're reporting results on CPU, please note this when you report.
  warnings.warn(
/Users/sterdam/opt/anaconda3/envs/photobook/lib/python3.8/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
  0%|                                                                                    | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.07s/it]
/Users/sterdam/cterdam/11711/photobook-full-listener/preprocess/verify_clipscore/../clipscore.py:123: UserWarning: due to a numerical instability, new numpy normalization is slightly different than paper results. to exactly replicate paper results, please use numpy version less than 1.21, e.g., 1.20.3.
  warnings.warn(


In [80]:
for x in result:
    print(x)

[0.64717317 0.5265468  0.5040432  0.5364773  0.4304402  0.4780367 ]
[0.48825443 0.7448214  0.5073978  0.5759537  0.4466368  0.5158828 ]
[0.41538388 0.53866017 0.7049305  0.5953658  0.40107253 0.45484954]
[0.41835874 0.4693898  0.5689244  0.7584378  0.4485127  0.4531071 ]
[0.41933477 0.41303173 0.5398673  0.52585316 0.6901908  0.394435  ]
[0.40232822 0.48830146 0.49188823 0.54526323 0.41508007 0.65607315]


# TorchMetrics implementation

In [43]:
metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch32")

In [45]:
def img2tensor(imgname):
    image = Image.open(imgname)
    transform = transforms.Compose([transforms.PILToTensor()])
    img_tensor = transform(image)
    return img_tensor

In [77]:
score = metric(img2tensor('./imgdir/inner/cat.jpg'), "This is a cat")

In [78]:
score.detach()

tensor(26.6762)